**Задание 1. Делаем размеченный тестсет. Будем пользоваться словарем ФЭБ и анализировать лексему *замок***

Значение 1. Дворец и крепость феодала. Диагностическое слово: принцесса

Значение 2. Устройство для запирания чего-л. ключом. Диагностическое слово: ключ

In [59]:
#открваем заранее подготовленные файлы с контекстами к каждому из значений 
f1 = open('Meaning1.txt', 'r', encoding = 'utf-8')
meaning1 = f1.read()
f1.close()
f2 = open('Meaning2.txt', 'r', encoding = 'utf-8')
meaning2 = f2.read()
f2.close()

**Задание 2. Определяем значение автоматически — с помощью AdaGram** 

In [2]:
import adagram
from nltk.tokenize import RegexpTokenizer

In [3]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')

In [4]:
#!pip install pymystem3

In [5]:
from pymystem3 import Mystem

In [6]:
import numpy as np
import pandas as pd

In [73]:
from nltk.corpus import stopwords
stops = set(stopwords.words('russian'))

In [7]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [57]:
def tokenize(text):
    return token.tokenize(text)

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

In [55]:
mystem = Mystem()

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return probs.argmax()

def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(", ".join(tokenize(s)))]

In [9]:
#создаем словарь, где ключ - номер значения слова "замок", значение - значение слова по ключу
dict_castle = {0: [('орденский', 0, 0.6297175),
 ('монсегюр', 0, 0.61552596),
 ('аббатство', 0, 0.6004313),
 ('дворец', 2, 0.5968682),
 ('сеньория', 0, 0.59168553),
 ('герцог', 2, 0.59005535),
 ('инстербург', 0, 0.57428056),
 ('каркассон', 0, 0.57125235),
 ('резиденция', 0, 0.5694655),
 ('амбуаз', 0, 0.56807244)], 1: [('задвижка', 0, 0.7730665),
 ('запор', 1, 0.7499841),
 ('засов', 1, 0.73436505),
 ('ключ', 0, 0.72064495),
 ('отмыкать', 0, 0.7102439),
 ('шпингалет', 0, 0.70520604),
 ('щеколда', 0, 0.70305103),
 ('защелка', 1, 0.70179313),
 ('запираться', 0, 0.696132),
 ('дверь', 6, 0.69068426)], 2: [('крепость', 1, 0.70243007),
 ('башня', 1, 0.6900965),
 ('бастион', 2, 0.66138613),
 ('средневековый', 1, 0.66027975),
 ('величественный', 1, 0.6602245),
 ('донжон', 0, 0.6288717),
 ('цитадель', 0, 0.62760615),
 ('башня', 3, 0.60489523),
 ('здание', 3, 0.6001993),
 ('крепостной', 0, 0.596294)], 3: [('крепость', 3, 0.60584474),
 ('дворец', 1, 0.5704742),
 ('башня', 5, 0.5693793),
 ('пещера', 0, 0.56734097),
 ('форт', 2, 0.56186247),
 ('холд', 1, 0.56155473),
 ('дом', 2, 0.5479651),
 ('подземелье', 0, 0.53780484),
 ('королевский', 2, 0.5256898),
 ('ругна', 0, 0.5172116)]}

In [10]:
word = 'замок'

In [60]:
mean1_array = []
for i in meaning1.split('.'): #замок - Дворец и крепость феодала
    if i != '  ':
        means1 = disambiguate(vm, word, i)
        mean1_array.append(means1)

In [61]:
mean1_array #получили номера значений для каждого контекста

[3, 3, 2, 3, 3, 3, 0, 3, 3, 2]

In [17]:
#теперь с помощью словаря по номерам значений находим значения для каждого номера
meanings_contexts1 = []
for i in mean1_array:
    for key in dict_castle:
        if i == key:
            meanings_contexts1.append(dict_castle[key])

In [18]:
meanings_contexts1 #значения для каждого контекста

[[('крепость', 3, 0.60584474),
  ('дворец', 1, 0.5704742),
  ('башня', 5, 0.5693793),
  ('пещера', 0, 0.56734097),
  ('форт', 2, 0.56186247),
  ('холд', 1, 0.56155473),
  ('дом', 2, 0.5479651),
  ('подземелье', 0, 0.53780484),
  ('королевский', 2, 0.5256898),
  ('ругна', 0, 0.5172116)],
 [('крепость', 3, 0.60584474),
  ('дворец', 1, 0.5704742),
  ('башня', 5, 0.5693793),
  ('пещера', 0, 0.56734097),
  ('форт', 2, 0.56186247),
  ('холд', 1, 0.56155473),
  ('дом', 2, 0.5479651),
  ('подземелье', 0, 0.53780484),
  ('королевский', 2, 0.5256898),
  ('ругна', 0, 0.5172116)],
 [('крепость', 1, 0.70243007),
  ('башня', 1, 0.6900965),
  ('бастион', 2, 0.66138613),
  ('средневековый', 1, 0.66027975),
  ('величественный', 1, 0.6602245),
  ('донжон', 0, 0.6288717),
  ('цитадель', 0, 0.62760615),
  ('башня', 3, 0.60489523),
  ('здание', 3, 0.6001993),
  ('крепостной', 0, 0.596294)],
 [('крепость', 3, 0.60584474),
  ('дворец', 1, 0.5704742),
  ('башня', 5, 0.5693793),
  ('пещера', 0, 0.56734097),
  

In [19]:
main_meanings_for_10_contexts_1 = []
for i in meanings_contexts1:
    main_meanings_for_10_contexts_1.append(i[0][0])

In [22]:
main_meanings_for_10_contexts_1 #из каждого набора значений для каждого контекста взяли первое значение (чтобы нагляднее)

['крепость',
 'крепость',
 'крепость',
 'крепость',
 'крепость',
 'крепость',
 'орденский',
 'крепость',
 'крепость',
 'крепость']

In [63]:
mean2_array = []
for i in meaning2.split('.'): #замок - Устройство для запирания чего-л. ключом.
    if i != '':
        means2 = disambiguate(vm, word, i)
        mean2_array.append(means2)

In [64]:
mean2_array #получили номера значений для каждого контекста

[1, 1, 1, 1, 3, 1, 1, 1, 3, 1]

In [29]:
#теперь с помощью словаря по номерам значений находим занчения для каждого номера
meanings_contexts2 = [] 
for i in mean2_array:
    for key in dict_castle:
        if i == key:
            meanings_contexts2.append(dict_castle[key])

In [30]:
meanings_contexts2

[[('задвижка', 0, 0.7730665),
  ('запор', 1, 0.7499841),
  ('засов', 1, 0.73436505),
  ('ключ', 0, 0.72064495),
  ('отмыкать', 0, 0.7102439),
  ('шпингалет', 0, 0.70520604),
  ('щеколда', 0, 0.70305103),
  ('защелка', 1, 0.70179313),
  ('запираться', 0, 0.696132),
  ('дверь', 6, 0.69068426)],
 [('задвижка', 0, 0.7730665),
  ('запор', 1, 0.7499841),
  ('засов', 1, 0.73436505),
  ('ключ', 0, 0.72064495),
  ('отмыкать', 0, 0.7102439),
  ('шпингалет', 0, 0.70520604),
  ('щеколда', 0, 0.70305103),
  ('защелка', 1, 0.70179313),
  ('запираться', 0, 0.696132),
  ('дверь', 6, 0.69068426)],
 [('задвижка', 0, 0.7730665),
  ('запор', 1, 0.7499841),
  ('засов', 1, 0.73436505),
  ('ключ', 0, 0.72064495),
  ('отмыкать', 0, 0.7102439),
  ('шпингалет', 0, 0.70520604),
  ('щеколда', 0, 0.70305103),
  ('защелка', 1, 0.70179313),
  ('запираться', 0, 0.696132),
  ('дверь', 6, 0.69068426)],
 [('задвижка', 0, 0.7730665),
  ('запор', 1, 0.7499841),
  ('засов', 1, 0.73436505),
  ('ключ', 0, 0.72064495),
  ('от

In [31]:
main_meanings_for_10_contexts_2 = []
for i in meanings_contexts2:
    main_meanings_for_10_contexts_2.append(i[0][0])

In [33]:
main_meanings_for_10_contexts_2 #из каждого набора значений для каждого контекста взяли первое значение (чтобы нагляднее)

['задвижка',
 'задвижка',
 'задвижка',
 'задвижка',
 'крепость',
 'задвижка',
 'задвижка',
 'задвижка',
 'крепость',
 'задвижка']

**Задание 3. Припишем значение толкованию**

In [34]:
interpretation1 = "Дворец и крепость феодала"
interpretation2 = "Устройство для запирания чего-л. ключом"

In [36]:
#подбираем номер значения для каждого определения
means_intr1 = disambiguate(vm, word, interpretation1)
means_intr2 = disambiguate(vm, word, interpretation2)

In [229]:
#для первого определения получилось такое значение
for key in dict_castle:
    if key == means_intr1:
        print(dict_castle[key])

[('орденский', 0, 0.6297175), ('монсегюр', 0, 0.61552596), ('аббатство', 0, 0.6004313), ('дворец', 2, 0.5968682), ('сеньория', 0, 0.59168553), ('герцог', 2, 0.59005535), ('инстербург', 0, 0.57428056), ('каркассон', 0, 0.57125235), ('резиденция', 0, 0.5694655), ('амбуаз', 0, 0.56807244)]


In [37]:
#для второго определения получилось такое значение
for key in dict_castle:
    if key == means_intr2:
        print(dict_castle[key])

[('задвижка', 0, 0.7730665), ('запор', 1, 0.7499841), ('засов', 1, 0.73436505), ('ключ', 0, 0.72064495), ('отмыкать', 0, 0.7102439), ('шпингалет', 0, 0.70520604), ('щеколда', 0, 0.70305103), ('защелка', 1, 0.70179313), ('запираться', 0, 0.696132), ('дверь', 6, 0.69068426)]


**Задание 4. WikiWordnet. Определяем толкования для контекстов**

In [38]:
#!pip install wiki_ru_wordnet

In [39]:
from wiki_ru_wordnet import WikiWordnet
import re
ruwn = WikiWordnet()

In [43]:
s = ruwn.get_synsets('замок')

In [45]:
#словарь всех определений слова "замок"
definitions = []
for i in s:
    for w in i.get_words():
        definitions.append(w.definition())

In [93]:
from rank_bm25 import BM25Okapi
tokenized_definitions = [doc.split(" ") for doc in definitions]
bm25 = BM25Okapi(tokenized_definitions)

In [122]:
#для каждого контекста смотрим bm25 с каждым определением и находим индекс лучшего определения для каждого контекста
best_indxs = []
for query in meaning1.split('.'):
    if query != '  ':
        tokenized_query = query.split(" ")
        doc_scores = bm25.get_scores(tokenized_query)
        best = np.where(doc_scores == np.amax(doc_scores))
        print('List of Indices of maximum element :', best[0])
        best_indxs.append(best[0][0]) #список индексов самых подходящих определений

List of Indices of maximum element : [4]
List of Indices of maximum element : [5]
List of Indices of maximum element : [4]
List of Indices of maximum element : [5]
List of Indices of maximum element : [5]
List of Indices of maximum element : [1]
List of Indices of maximum element : [1]
List of Indices of maximum element : [5]
List of Indices of maximum element : [2]
List of Indices of maximum element : [4]


In [129]:
auto_def1 = []
for i in best_indxs:
    auto_def1.append(definitions[i])
    print(definitions[i])
    print()

замок~ru~замок~ru~[[устройство]] для запирания чего-либо [[ключ]]ом {{пример|Дверной {{выдел|замок}}.}} {{пример|Запереть дверь на {{выдел|замок}}.}} {{пример|Ключ в {{выдел|замке}}.}} {{пример|Каморка запиралась на {{выдел|замо́к}}, напоминавший своим видом калач, только чёрный; ключ от этого {{выдел|замка́}} Герасим всегда носил с собой на пояске.|Тургенев|[[:s:Муму (Тургенев)|Муму]]|1852|источник=source}} {{пример|Ключ заскрипел в заржавленном {{выдел|замке}}, и дверь отворилась; им в лицо пахнуло сыростью.|Лермонтов|[[:s:Штосс (Лермонтов)|Штосс]]|1841|источник=source}}~57866~29025

запор~ru~запор~ru~[[устройство]], предназначенное для запирания чего-либо {{пример|Князь снял {{выдел|запор}}, отворил дверь и ― отступил в изумлении, весь даже вздрогнул: пред ним стояла Настасья Филипповна.|Ф. М. Достоевский|Идиот|1869|источник=НКРЯ}} {{пример|Потом он суёт руку под подушку и протягивает мне похожий на часы блестящий никелированный компас с крышкой, с {{выдел|запором}} и с вертящейся ф

In [126]:
#делаем все то же самое для контекстов, где "замо`к" 
best_indxs2 = []
for query in meaning2.split('.'):
    if query != '':
        tokenized_query2 = query.split(" ")
        doc_scores2 = bm25.get_scores(tokenized_query2)
        best2 = np.where(doc_scores2 == np.amax(doc_scores2))
        print('List of Indices of maximum element :', best2[0])
        best_indxs2.append(best2[0][0]) #список индексов самых подходящих определений

List of Indices of maximum element : [4]
List of Indices of maximum element : [5]
List of Indices of maximum element : [4]
List of Indices of maximum element : [1]
List of Indices of maximum element : [4]
List of Indices of maximum element : [5]
List of Indices of maximum element : [4]
List of Indices of maximum element : [4]
List of Indices of maximum element : [4]
List of Indices of maximum element : [5]


In [128]:
auto_def2 = []
for i in best_indxs2:
    auto_def2.append(definitions[i])
    print(definitions[i])
    print()

замок~ru~замок~ru~[[устройство]] для запирания чего-либо [[ключ]]ом {{пример|Дверной {{выдел|замок}}.}} {{пример|Запереть дверь на {{выдел|замок}}.}} {{пример|Ключ в {{выдел|замке}}.}} {{пример|Каморка запиралась на {{выдел|замо́к}}, напоминавший своим видом калач, только чёрный; ключ от этого {{выдел|замка́}} Герасим всегда носил с собой на пояске.|Тургенев|[[:s:Муму (Тургенев)|Муму]]|1852|источник=source}} {{пример|Ключ заскрипел в заржавленном {{выдел|замке}}, и дверь отворилась; им в лицо пахнуло сыростью.|Лермонтов|[[:s:Штосс (Лермонтов)|Штосс]]|1841|источник=source}}~57866~29025

запор~ru~запор~ru~[[устройство]], предназначенное для запирания чего-либо {{пример|Князь снял {{выдел|запор}}, отворил дверь и ― отступил в изумлении, весь даже вздрогнул: пред ним стояла Настасья Филипповна.|Ф. М. Достоевский|Идиот|1869|источник=НКРЯ}} {{пример|Потом он суёт руку под подушку и протягивает мне похожий на часы блестящий никелированный компас с крышкой, с {{выдел|запором}} и с вертящейся ф

**Задание 5. Определяем, совпадают ли автоматически определенные значения с автоматически определенными толкованиями**

In [131]:
from sklearn.metrics import accuracy_score

In [137]:
talk1 = []
for i in auto_def1:
    norm1 = normalize_pm(i)[0]
    talk1.append(norm1)

In [138]:
accuracy_score(main_meanings_for_10_contexts_1, talk1)

0.1

In [139]:
talk2 = []
for i in auto_def2:
    norm2 = normalize_pm(i)[0]
    talk2.append(norm2)

In [140]:
accuracy_score(main_meanings_for_10_contexts_2, talk2)

0.0

На самом деле, accuracy_score был бы лучше, если бы рассматривали не только первые значения (main_meanings_for_10_contexts_1(2)), а все. Но дедлайн есть дедлайн :)